In [5]:
import pandas as pd
from sqlalchemy import create_engine
from getpass import getpass



In [6]:
def connect_to_sakila(username, password, host, port, database):
    username = "root"
    password = getpass()
    host = "127.0.0.1"
    port = 3306
    database = "sakila"
    

    connection_str = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"


    engine = create_engine(connection_str)

    return engine

In [7]:
def rentals_month(engine, month, year):

    query = f"""
    SELECT customer_id, COUNT(*) AS rental_count
    FROM rental
    WHERE YEAR(rental_date) = {year} AND MONTH(rental_date) = {month}
    GROUP BY customer_id
    """

    
    rental_data = pd.read_sql_query(query, engine)

    return rental_data

In [8]:
def rental_count_month(rental_data, month, year):

    column_name = f"rentals_{month}_{year}"
    rental_data.rename(columns={'rental_count': column_name}, inplace=True)

    return rental_data

In [9]:
def compare_rentals(rental_data_month1, rental_data_month2):
    
    comparison_df = pd.merge(rental_data_month1, rental_data_month2, on='customer_id', how='outer')
    
    comparison_df.fillna(0, inplace=True)
    
    comparison_df['difference'] = comparison_df.iloc[:, 1] - comparison_df.iloc[:, 2]

    return comparison_df

In [10]:
engine = connect_to_sakila("username", "password", "localhost", 3306, "sakila")